# Error analysis for the type of examples that Stop words get wrong compared to the baseline

In [3]:
%load_ext autoreload

In [4]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.converters.converter_ddi import relation_dict
output_path = '/scratch/geeticka/relation-extraction-result/ddi-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/scratch/geeticka/relation-extraction-result/ddi-analyze/test_original.txt')
stop_words_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/ddi/pre-processed/punct_stop_digit/test_punct_stop_digit.txt')

In [5]:
relation_dict

{0: 'advise', 1: 'effect', 2: 'mechanism', 3: 'int', 4: 'none'}

In [6]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [7]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [8]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [9]:
needed_linenum_and_relation = {}

In [10]:
with open(res("answers_for_dev-stop-word.txt")) as textfile1, open(res('answers_for_dev-baseline.txt')) as textfile2, \
open(res('answers_for_dev-baseline_gold.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, stop_words_relation = read_answers_line(x)
        _, baseline_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if baseline_relation == gold_relation and stop_words_relation != gold_relation: # punct digit making mistakes
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, stop_words_relation)

In [11]:
len(list(needed_linenum_and_relation.keys()))

267

Remember that baseline is correct here so the first relation listed is the correct one, unlike in the i2b2 notebook where the order is reversed because baseline is the incorrect one there

In [12]:
interaction = 0; mechanism = 0; advice = 0; effect = 0; none = 0

In [13]:
print('We print the baseline first and then the stop words version. Gold relation corresponds to baseline\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(stop_words_sentences_path) as stop_words_sentences:
    for x, y in zip(original_sentences, stop_words_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            if r_b == 'int': interaction += 1
            elif r_b == 'mechanism': mechanism += 1
            elif r_b == 'advise': advice += 1
            elif r_b == 'effect': effect += 1
            elif r_b == 'none': none += 1
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the stop words version. Gold relation corresponds to baseline


Predicted Relation: 	 none, mechanism 
Entities: 	 terbinafine, terbinafine 	 tolbutamide, tolbutamide 
Sentences: 
	In vitro studies with human liver microsomes showed that terbinafine does not inhibit the metabolism of tolbutamide , ethinylestradiol , ethoxycoumarin , and cyclosporine . 
	 vitro studies human liver microsomes showed terbinafine inhibit metabolism tolbutamide ethinylestradiol ethoxycoumarin cyclosporine


Predicted Relation: 	 none, mechanism 
Entities: 	 terbinafine, terbinafine 	 ethinylestradiol, ethinylestradiol 
Sentences: 
	In vitro studies with human liver microsomes showed that terbinafine does not inhibit the metabolism of tolbutamide , ethinylestradiol , ethoxycoumarin , and cyclosporine . 
	 vitro studies human liver microsomes showed terbinafine inhibit metabolism tolbutamide ethinylestradiol ethoxycoumarin cyclosporine


Predicted Relation: 	 none, mechani

In [14]:
print(interaction, mechanism, advice, effect, none)

14 52 29 37 135


In [15]:
interaction + mechanism + advice + effect + none

267